<a href="https://colab.research.google.com/github/ufbfung/icd10/blob/main/icd10_apis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Querying ICD-10 codes
The WHO provides an API to programmatically access ICD codes, but it's primarily targeted at ICD-11 as the access to ICD-10 codes is extremely limited. Because of this, this notebook will explore other APIs that can access ICD-10 codes.

One example is: https://clinicaltables.nlm.nih.gov/apidoc/icd10cm/v3/doc.html

Some example API queries we will build in this notebook:
Sample API Queries

|Query|Result|Description|
|--|--|--|
|https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search?sf=code,name&terms=tuberc|[71,["A15.0","A15.4","A15.5","A15.6","A15.7","A15.8","A15.9"],null,[["A15.0","Tuberculosis of lung"], ["A15.4","Tuberculosis of intrathoracic lymph nodes"],["A15.5","Tuberculosis of larynx, trachea and bronchus"],["A15.6","Tuberculous pleurisy"], ["A15.7","Primary respiratory tuberculosis"],["A15.8","Other respiratory tuberculosis"],["A15.9","Respiratory tuberculosis unspecified"]]]|Returns a complete count of the 71 diagnoses that match the string "tuberc", displaying both the ICD-10-CM code and its associated term for the first 7 terms.
|https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search?sf=code,name&terms=A15|[7,["A15.0","A15.4","A15.5","A15.6","A15.7","A15.8","A15.9"],null,[["A15.0","Tuberculosis of lung"],["A15.4","Tuberculosis of intrathoracic lymph nodes"],["A15.5","Tuberculosis of larynx, trachea and bronchus"],["A15.6","Tuberculous pleurisy"],["A15.7","Primary respiratory tuberculosis"],["A15.8","Other respiratory tuberculosis"],["A15.9","Respiratory tuberculosis unspecified"]]]|Returns a list of all ICD-10-CM codes beginning with "A15" and their associated terms.|

In [21]:
# Import relevant libraries
import requests
import json
import pandas as pd
import os

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Set paths
root_path = '/content/drive/MyDrive/coding'
icd10_path = os.path.join(root_path,'icd10')

# Set files
icd10_csv = os.path.join(icd10_path,'icd10_t1dm.csv')

In [31]:
# Read CSV into dataframe
df = pd.read_csv(os.path.join(icd10_path,'icd10_t1dm.csv'))

# Extract 'Code' column
code_column = df['Code']

# Display the 'Code' column
print(code_column)

0     E1010
1     E1011
2     E1021
3     E1022
4     E1029
      ...  
82     E108
83     E109
84      E10
85    99091
86    99457
Name: Code, Length: 87, dtype: object


In [35]:
# Set base endpoint and search parameters
base_url = 'https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search'
base_param = '?sf=code,name&terms='
icd10_code = 'A15'

# Construct query
query = base_url + base_param + icd10_code
print(query)

# Make request
response = requests.get(query)
if response.status_code == 200:
    data = json.loads(response.text)
    if isinstance(data, list) and len(data) >= 4:
        results = data[3]
        if isinstance(results, list) and len(results) > 0:
            df = pd.DataFrame(results, columns=['Code', 'Name'])
            print(df)
        else:
            print(f'No information found for ICD-10 code: {icd10_code}')
    else:
        print(f'Invalid response for ICD-10 code: {icd10_code}')
else:
    print(f'Error retrieving information for ICD-10 code: {icd10_code}')

https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search?sf=code,name&terms=A15
    Code                                          Name
0  A15.0                          Tuberculosis of lung
1  A15.4     Tuberculosis of intrathoracic lymph nodes
2  A15.5  Tuberculosis of larynx, trachea and bronchus
3  A15.6                          Tuberculous pleurisy
4  A15.7              Primary respiratory tuberculosis
5  A15.8                Other respiratory tuberculosis
6  A15.9          Respiratory tuberculosis unspecified


In [36]:
base_url = 'https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search'
base_param = '?sf=code,name&terms='
icd10_codes = ['A15', 'B20']

for icd10_code in icd10_codes:
    query = base_url + base_param + icd10_code
    response = requests.get(query)

    if response.status_code == 200:
        data = json.loads(response.text)
        if isinstance(data, list) and len(data) >= 4:
            codes = data[1]
            descriptions = data[3]

            if isinstance(codes, list) and isinstance(descriptions, list):
                for code, description in zip(codes, descriptions):
                    print(f'ICD-10-CM Code: {code}')
                    print(f'Description: {description}')
                    print('---')
            else:
                print(f'No information found for ICD-10 code: {icd10_code}')
        else:
            print(f'Invalid response for ICD-10 code: {icd10_code}')
    else:
        print(f'Error retrieving information for ICD-10 code: {icd10_code}')

ICD-10-CM Code: A15.0
Description: ['A15.0', 'Tuberculosis of lung']
---
ICD-10-CM Code: A15.4
Description: ['A15.4', 'Tuberculosis of intrathoracic lymph nodes']
---
ICD-10-CM Code: A15.5
Description: ['A15.5', 'Tuberculosis of larynx, trachea and bronchus']
---
ICD-10-CM Code: A15.6
Description: ['A15.6', 'Tuberculous pleurisy']
---
ICD-10-CM Code: A15.7
Description: ['A15.7', 'Primary respiratory tuberculosis']
---
ICD-10-CM Code: A15.8
Description: ['A15.8', 'Other respiratory tuberculosis']
---
ICD-10-CM Code: A15.9
Description: ['A15.9', 'Respiratory tuberculosis unspecified']
---
ICD-10-CM Code: B20
Description: ['B20', 'Human immunodeficiency virus [HIV] disease']
---
